<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/Real_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar las librerías necesarias
!pip install opencv-python
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install torch torchvision

# Descargar los pesos pre-entrenados del modelo SAM
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

# Importar las librerías
import cv2
import numpy as np
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
import matplotlib.pyplot as plt
from google.colab.output import eval_js
from IPython.display import display, Javascript
from base64 import b64decode
%matplotlib inline

# Configurar el modelo SAM
device = "cuda" if torch.cuda.is_available() else "cpu"
model_type = "vit_b"
sam_checkpoint = "sam_vit_b_01ec64.pth"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
mask_generator = SamAutomaticMaskGenerator(sam)

# Descargar el filtro (ojos de gato)
!wget https://i.imgur.com/8fj6G9M.png -O cat_eyes.png

# Cargar el filtro
cat_eyes = cv2.imread('cat_eyes.png', cv2.IMREAD_UNCHANGED)

# Función para capturar imagen desde la webcam
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
        const div = document.createElement('div');
        const video = document.createElement('video');
        const btn = document.createElement('button');
        btn.textContent = 'Tomar foto';
        div.appendChild(video);
        div.appendChild(btn);
        document.body.appendChild(div);

        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        video.srcObject = stream;
        await video.play();

        // Esperar a que el usuario haga clic
        await new Promise((resolve) => btn.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    img = cv2.imread(filename)
    return img

# Capturar la imagen
frame = take_photo()

# Convertir la imagen a RGB
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# Generar las máscaras
masks = mask_generator.generate(frame_rgb)

# Seleccionar la máscara más grande
if len(masks) > 0:
    masks = sorted(masks, key=lambda x: x['area'], reverse=True)
    mask = masks[0]['segmentation']
else:
    mask = np.zeros(frame.shape[:2], dtype=bool)

# Superponer el filtro en la región segmentada
contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if contours:
    x, y, w, h = cv2.boundingRect(contours[0])
    filter_resized = cv2.resize(cat_eyes, (w, int(cat_eyes.shape[0] * w / cat_eyes.shape[1])))
    fh, fw = filter_resized.shape[:2]
    x_offset = x
    y_offset = y + int(h/4)  # Ajuste vertical
    if y_offset + fh > frame.shape[0]:
        fh = frame.shape[0] - y_offset
        filter_resized = filter_resized[:fh]
    if x_offset + fw > frame.shape[1]:
        fw = frame.shape[1] - x_offset
        filter_resized = filter_resized[:, :fw]
    # Superponer el filtro
    alpha_s = filter_resized[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s
    for c in range(0, 3):
        frame[y_offset:y_offset+fh, x_offset:x_offset+fw, c] = (alpha_s * filter_resized[:, :, c] + alpha_l * frame[y_offset:y_offset+fh, x_offset:x_offset+fw, c])

# Mostrar el resultado
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()